# **Naive Bayes — Módulo 3, Notebook 2/6**

---

## Índice

1. [Introdução ao Naive Bayes](#introducao)
2. [Como o Naive Bayes Funciona?](#como-funciona)
3. [Intuição Prática](#intuicao)
4. [Formalização Matemática](#formalizacao)
5. [Tipos de Naive Bayes](#tipos)
6. [Implementação com Scikit-Learn](#sklearn)
7. [Vantagens e Desvantagens](#vantagens-desvantagens)

---

<a id='introducao'></a>
## **Introdução ao Naive Bayes**

Imagine que acabamos de receber um email. O sistema precisa decidir se essa mensagem, com o assunto "Esse está pagando! Aposte agora", é um email importante ou spam. Como podemos resolver esse problema automaticamente?

Esse é um cenário clássico para o algoritmo **Naive Bayes**, um dos métodos mais elegantes e eficientes em machine learning. Diferentemente do KNN que memoriza todos os dados, o Naive Bayes aprende padrões estatísticos para tomar decisões probabilísticas.

O Naive Bayes é um classificador probabilístico baseado no **Teorema de Bayes**. Na classificação Bayesiana, estamos interessados em calcular a probabilidade de certa observação pertencer a uma categoria (ou classe) dado as características observadas. Matematicamente, queremos encontrar $P(C_k | X)$ — a probabilidade da categoria $k$ dado o vetor de características $X$.

<a id='como-funciona'></a>
## **Como o Naive Bayes Funciona?**

O algoritmo Naive Bayes classifica uma amostra $X = (x_1, x_2, ..., x_n)$ atribuindo a classe $C_k$ que maximiza a probabilidade condicional:

$$P(C_k|X) = \frac{P(X|C_k)P(C_k)}{P(X)}$$

Onde:
- $P(C_k|X)$: **Probabilidade a posteriori** — probabilidade da classe $C_k$ dados os atributos $X$
- $P(X|C_k)$: **Verossimilhança** — probabilidade de observar os atributos $X$ na classe $C_k$
- $P(C_k)$: **Probabilidade a priori** — probabilidade prévia da classe $C_k$
- $P(X)$: **Evidência** — probabilidade dos atributos $X$ (constante para todas as classes)

A predição é feita escolhendo a classe que maximiza $P(C_k|X)$.

### **De onde vem o "Naive" (ingênuo)?**

O termo "naive" refere-se à principal **hipótese simplificadora**: as variáveis são independentes entre si, dada a classe. Com essa suposição, a probabilidade condicional se torna:

$$P(X|C_k) = P(x_1|C_k) \times P(x_2|C_k) \times ... \times P(x_n|C_k)$$

Isso facilita enormemente os cálculos, pois agora basta calcular a probabilidade de cada feature separadamente e multiplicá-las.

<a id='intuicao'></a>
## **Intuição Prática: Classificando Emails como Spam**

Vamos consolidar esse conhecimento através de um exemplo prático. Utilizaremos o **SMS Spam Collection Dataset** para entender como o Naive Bayes toma decisões.

### **O Problema**

Recebemos um SMS com o texto:

**"WINNER!! You have won a £1000 prize! Call now to claim!"**

**Desafio:** Classificar essa mensagem como `spam` ou `ham` (não spam).

Para resolver isso, o Naive Bayes segue três etapas fundamentais, que implementaremos passo a passo.

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [ ]:
# Carregando o SMS Spam Collection Dataset
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "uciml/sms-spam-collection-dataset",
    "spam.csv",
    pandas_kwargs={"encoding": "latin", "usecols": [0, 1], "names": ["class", "sms"], 'header': 0}
)

print(f"Total de mensagens: {len(df)}")
print(f"\nPrimeiras 5 mensagens:")
df.head()

### **Pré-processamento de Texto**

Antes de aplicar o Naive Bayes, precisamos processar o texto. Vamos criar uma função que:

1. Transforma todo o texto em minúsculo
2. Remove pontuações
3. Tokeniza (separa as palavras)
4. Remove stopwords (palavras comuns como "e", "o") — opcional, mas boa prática

In [ ]:
def preprocess_text(text):
    """
    Pré-processa o texto removendo pontuações, convertendo para minúsculo e tokenizando.
    
    Parâmetros:
    text (str): Texto a ser processado
    
    Retorna:
    list: Lista de palavras processadas
    """
    # Converter para minúsculo
    text = text.lower()
    
    # Remover pontuações e caracteres especiais
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenizar (separar em palavras)
    words = text.split()
    
    # Stopwords básicas em inglês (opcional - simplificado para o exemplo)
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'is', 'are', 'was', 'were'}
    
    # Remover stopwords
    words = [word for word in words if word not in stopwords]
    
    return words

# Aplicando o pré-processamento
df['words'] = df['sms'].apply(preprocess_text)

print("Exemplo de pré-processamento:")
print(f"Original: {df['sms'].iloc[0]}")
print(f"Processado: {df['words'].iloc[0]}")

In [ ]:
# ===================================================================
# PASSO 1: CALCULAR PROBABILIDADES A PRIORI
# Fórmula: P(class) = número de mensagens da classe / total de mensagens
# ===================================================================

total_messages = len(df)
spam_count = len(df[df['class'] == 'spam'])
ham_count = len(df[df['class'] == 'ham'])

prior_spam = spam_count / total_messages
prior_ham = ham_count / total_messages

print(f"Total de Mensagens: {total_messages}")
print(f"Total de Mensagens Spam: {spam_count}")
print(f"Total de Mensagens Ham: {ham_count}")
print(f"\nP(SPAM): {prior_spam*100:.2f}%")
print(f"P(HAM): {prior_ham*100:.2f}%")

print(f"\nInterpretação:")
print(f"  Antes de analisar o conteúdo de uma mensagem:")
print(f"  - Probabilidade de ser SPAM: {prior_spam*100:.2f}%")
print(f"  - Probabilidade de ser HAM: {prior_ham*100:.2f}%")

In [ ]:
# ===================================================================
# PASSO 2: CALCULAR VEROSSIMILHANÇA (Likelihood)
# Fórmula: P(word|class) = (count(word in class) + 1) / (total words in class + V)
# Onde V é o tamanho do vocabulário (número de palavras únicas)
# O "+1" é a suavização de Laplace — evita probabilidade zero
# ===================================================================

spam_words = df[df['class'] == 'spam']['words'].sum()
ham_words = df[df['class'] == 'ham']['words'].sum()

# Contar a frequência das palavras em cada classe
spam_word_count = Counter(spam_words)
ham_word_count = Counter(ham_words)

vocabulary = set(spam_word_count.keys()).union(set(ham_word_count.keys()))

total_spam_words = len(spam_words)
total_ham_words = len(ham_words)

print(f"Estatísticas de Palavras:\n")
print(f"  - Vocabulário Total (V): {len(vocabulary)} palavras únicas")
print(f"  - Total de Palavras em Mensagens SPAM: {total_spam_words}")
print(f"  - Total de Palavras em Mensagens HAM: {total_ham_words}")

print(f"\nPalavras mais comuns em SPAM:")
for word, count in spam_word_count.most_common(5):
    print(f"  {word}: {count}")
    
print(f"\nPalavras mais comuns em HAM:")
for word, count in ham_word_count.most_common(5):
    print(f"  {word}: {count}")

def likelihood(word, cls):
    """Calcula P(word|class) com suavização de Laplace"""
    if cls == 'spam':
        word_count = spam_word_count.get(word, 0)
        return (word_count + 1) / (total_spam_words + len(vocabulary))
    else:
        word_count = ham_word_count.get(word, 0)
        return (word_count + 1) / (total_ham_words + len(vocabulary))

print(f"\nExemplo de Cálculo de Verossimilhança:")
print(f"  P('free'|SPAM) = {likelihood('free', 'spam')*100:.4f}%")
print(f"  P('free'|HAM) = {likelihood('free', 'ham')*100:.4f}%")

In [ ]:
# ===================================================================
# PASSO 3: CLASSIFICAÇÃO DE UMA NOVA MENSAGEM
# ===================================================================

new_message = "WINNER!! You have won a £1000 prize! Call now to claim!"

preprocessed_message = preprocess_text(new_message)

print(f"Mensagem original: {new_message}")
print(f"Mensagem processada: {preprocessed_message}\n")

# Calcular probabilidade logarítmica para evitar underflow
# (multiplicar muitas probabilidades pequenas pode resultar em zero)
log_prob_spam = np.log(prior_spam)
log_prob_ham = np.log(prior_ham)

for word in preprocessed_message:
    log_prob_spam += np.log(likelihood(word, 'spam'))
    log_prob_ham += np.log(likelihood(word, 'ham'))

resultado = "SPAM" if log_prob_spam > log_prob_ham else "HAM"

print(f"Log P(SPAM|mensagem): {log_prob_spam:.4f}")
print(f"Log P(HAM|mensagem): {log_prob_ham:.4f}")
print(f"\nClassificação: {resultado}")

### **Explicação dos Três Passos**

**1. Probabilidade a Priori**

A probabilidade a priori é nossa crença sobre algo antes de analisar qualquer nova evidência:

- O algoritmo analisa todo o histórico de mensagens e calcula a proporção de cada categoria
- No nosso caso: $P(Spam) \approx 13\%$ e $P(Ham) \approx 87\%$
- Isso significa que, sem ler a nova mensagem, já sabemos que há uma chance baixa de ser spam

**2. Verossimilhança**

A verossimilhança mede o quão provável é encontrar a evidência (as palavras da mensagem) assumindo que ela pertence a uma determinada categoria:

- Se esta mensagem fosse Spam, qual seria a probabilidade de conter "winner", "prize", "call"?
- O algoritmo calcula a frequência dessas palavras dentro de cada categoria
- Como assumimos independência (naive), multiplicamos as probabilidades individuais

**3. Probabilidade a Posteriori**

A probabilidade a posteriori é o resultado final — a probabilidade de uma mensagem pertencer a uma categoria depois de analisarmos a evidência:

$$P(y|X) = \frac{P(X|y) \times P(y)}{P(X)}$$

Note que apesar da probabilidade a priori favorecer "Ham" (87%), a evidência contida nas palavras gera uma verossimilhança tão maior para "Spam" que a probabilidade a posteriori de ser Spam será mais alta!

<a id='formalizacao'></a>
## **Formalização Matemática**

Agora que você entendeu intuitivamente como o Naive Bayes funciona, vamos formalizar o algoritmo matematicamente.

### **Entrada do Algoritmo**

1. **Conjunto de treinamento:** $D = \{(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), ..., (x^{(n)}, y^{(n)})\}$

   Onde:
   - $x^{(i)}$: vetor de $d$ features ($x^{(i)} = (x^{(i)}_1, x^{(i)}_2, ..., x^{(i)}_d)$)
   - $y^{(i)}$: rótulo de classe pertencente ao conjunto finito $C = \{c_1, c_2, ..., c_k\}$

2. **Amostra para previsão:** $x^o = (x_1, x_2, ..., x_d)$ — vetor de features que precisa ser classificado

### **Processo do Algoritmo**

**Objetivo:** Encontrar a classe $\hat{y}$ que maximiza a probabilidade a posteriori $P(y|x^o)$

**1. Teorema de Bayes**

Para cada classe $c_k \in C$, calculamos a probabilidade a posteriori:

$$P(c_k|x^o) = \frac{P(x^o|c_k)P(c_k)}{P(x^o)}$$

Onde:
- $P(c_k|x^o)$: **Probabilidade a posteriori** — probabilidade da classe $c_k$ dado $x^o$
- $P(c_k)$: **Probabilidade a priori** — probabilidade da classe $c_k$
- $P(x^o|c_k)$: **Verossimilhança** — probabilidade da amostra $x^o$ dada a classe $c_k$
- $P(x^o)$: **Evidência** — probabilidade da amostra $x^o$ (constante para todas as classes)

**2. Cálculo da Probabilidade a Priori**

A priori é estimada a partir da frequência relativa de cada classe no conjunto de treinamento:

$$P(c_k) = \frac{\text{Número de amostras da classe } c_k}{\text{Número total de amostras}} = \frac{\sum_{i=1}^{n} I(y^{(i)} = c_k)}{n}$$

No código, implementamos isso calculando `spam_count / total_messages`.

**3. Cálculo da Verossimilhança (Suposição Naive)**

Aqui reside a suposição ingênua do algoritmo: todas as features são condicionalmente independentes, dada a classe:

$$P(x^o|c_k) = P(x_1, x_2, ..., x_d|c_k) = \prod^d_{j=1}P(x_j|c_k)$$

O cálculo de cada $P(x_j|c_k)$ depende da natureza da feature $x_j$, levando aos diferentes tipos de Naive Bayes.

**4. Tomada de Decisão (MAP — Maximum A Posteriori)**

Como $P(x^o)$ é constante para todas as classes, pode ser descartado na maximização:

$$\hat{y} = \arg \max_{c_k \in C} P(c_k) \prod^d_{j=1}P(x_j|c_k)$$

Onde $\hat{y} \in C$ é o rótulo da classe predita para $x^o$.

<a id='tipos'></a>
## **Tipos de Naive Bayes**

A principal diferença entre os tipos de Naive Bayes está em como eles modelam a distribuição $P(x_j|c_k)$ para diferentes tipos de dados.

### **1. Gaussian Naive Bayes**

**Uso:** Features contínuas que seguem uma distribuição normal (Gaussiana)

**Hipótese:** Para cada classe $c_k$, o valor da feature $x_j$ é distribuído segundo uma Gaussiana

**Processamento:** Durante o treinamento, calcula-se a média $\mu_{j,k}$ e a variância $\sigma^2_{j,k}$ da feature $j$ para todas as amostras da classe $c_k$. A verossimilhança é calculada usando a função de densidade de probabilidade (PDF) da distribuição normal:

$$P(x_j|c_k) = \frac{1}{\sqrt{2\pi\sigma^2_{j,k}}}\exp\left(-\frac{(x_j-\mu_{j,k})^2}{2\sigma^2_{j,k}}\right)$$

**Aplicações:** Classificação de dados numéricos contínuos (ex: temperatura, pressão, medições físicas)

### **2. Multinomial Naive Bayes**

**Uso:** Features discretas que representam contagens ou frequências (clássico para classificação de texto)

**Hipótese:** As features são geradas a partir de uma distribuição multinomial

**Processamento:** A verossimilhança $P(x_j|c_k)$ é a frequência relativa da feature $x_j$ (ex: uma palavra) dentro dos documentos da classe $c_k$. Para evitar probabilidade zero, usa-se a suavização de Laplace:

$$P(x_j|c_k) = \frac{N_{j,k} + \alpha}{N_k + \alpha d}$$

Onde:
- $N_{j,k}$: contagem da feature $x_j$ na classe $c_k$
- $N_k$: contagem total de todas as features na classe $c_k$
- $d$: número total de features únicas (tamanho do vocabulário)
- $\alpha$: parâmetro de suavização ($\alpha = 1$ para Laplace smoothing)

No código, implementamos isso com `(word_count + 1) / (total_words + len(vocabulary))`.

**Aplicações:** Classificação de texto, análise de sentimento, detecção de spam

### **3. Bernoulli Naive Bayes**

**Uso:** Features binárias (presença ou ausência de uma característica)

**Hipótese:** As features são variáveis binárias independentes

**Processamento:** A verossimilhança é calculada para a presença ($x_j = 1$) ou ausência ($x_j = 0$) da feature na classe $c_k$:

$$P(x_j|c_k) = P(j|c_k)^{x_j}(1-P(j|c_k))^{(1-x_j)}$$

Onde $P(j|c_k)$ é a probabilidade da feature $j$ ocorrer na classe $c_k$.

**Aplicações:** Classificação de documentos curtos com features binárias (ex: presença/ausência de palavras)

<a id='sklearn'></a>
## **Implementação com Scikit-Learn**

Na prática, usamos bibliotecas otimizadas como o Scikit-Learn. Vamos ver como é simples implementar o mesmo algoritmo:

In [ ]:
# Dados para Teste (Spam e Ham)
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split

df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "uciml/sms-spam-collection-dataset",
  "spam.csv",
  pandas_kwargs={"encoding": "latin", "usecols": [0,1], "names": ["class", "sms"], 'header': 0}
)

# Preparando os dados
X = df['sms'].tolist()
y = df['class'].tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Conjunto de treino: {len(X_train)} mensagens")
print(f"Conjunto de teste: {len(X_test)} mensagens")

### **Implementação Básica**

In [ ]:
# Passo 1: Vetorização (transformar texto em números)
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"Vocabulário (amostra): {list(vectorizer.get_feature_names_out())[1000:1005]}")
print(f"X_train_vec shape: {X_train_vec.shape}")
print(f"X_test_vec shape: {X_test_vec.shape}")

# Passo 2: Treinar o modelo
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)

# Passo 3: Fazer previsões
y_pred = nb.predict(X_test_vec)

# Passo 4: Avaliar
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia: {accuracy*100:.2f}%")

### **Técnicas Avançadas**

Vamos explorar variações e otimizações do Naive Bayes:

In [ ]:
# ===================================================================
# TÉCNICA 1: Comparando as Variantes do Naive Bayes
# ===================================================================

# 1. MultinomialNB (dados de contagem)
nb_multi = MultinomialNB()
nb_multi.fit(X_train_vec, y_train)
accuracy_multi = nb_multi.score(X_test_vec, y_test)

# 2. BernoulliNB (dados binários)
X_train_bin = (X_train_vec > 0).astype(int)
X_test_bin = (X_test_vec > 0).astype(int)

nb_bernoulli = BernoulliNB()
nb_bernoulli.fit(X_train_bin, y_train)
accuracy_bernoulli = nb_bernoulli.score(X_test_bin, y_test)

# 3. GaussianNB (dados contínuos)
X_train_dense = X_train_vec.toarray()
X_test_dense = X_test_vec.toarray()

nb_gaussian = GaussianNB()
nb_gaussian.fit(X_train_dense, y_train)
accuracy_gaussian = nb_gaussian.score(X_test_dense, y_test)

print("Comparação de Variantes do Naive Bayes:\n")
print(f"1. MultinomialNB: {accuracy_multi*100:.2f}% - Melhor para contagem de palavras")
print(f"2. BernoulliNB: {accuracy_bernoulli*100:.2f}% - Melhor para features binárias")
print(f"3. GaussianNB: {accuracy_gaussian*100:.2f}% - Melhor para dados contínuos")

In [ ]:
# ===================================================================
# TÉCNICA 2: Ajuste do Parâmetro Alpha (Suavização de Laplace)
# ===================================================================

# Parâmetros:
# - alpha: parâmetro de suavização de Laplace (default=1.0)
#   - alpha=1.0 é "add-one smoothing"
#   - alpha=0.0 significa sem suavização
# - fit_prior: se True, aprende probabilidade a priori dos dados
# - class_prior: probabilidades a priori customizadas

alphas = [0.001, 0.1, 0.5, 1.0, 2.0, 5.0]
results = []

print("Testando diferentes valores de alpha:\n")
for alpha in alphas:
    nb = MultinomialNB(alpha=alpha)
    nb.fit(X_train_vec, y_train)
    accuracy = nb.score(X_test_vec, y_test)
    results.append((alpha, accuracy))
    print(f"Alpha: {alpha:5.3f} → Acurácia: {accuracy*100:.2f}%")

best_alpha, best_acc = max(results, key=lambda x: x[1])
print(f"\nMelhor alpha: {best_alpha} com acurácia de {best_acc*100:.2f}%")

In [ ]:
# ===================================================================
# TÉCNICA 3: CountVectorizer vs TF-IDF Vectorizer
# ===================================================================

# CountVectorizer: conta a frequência das palavras
# TF-IDF: considera frequência + importância (palavras raras têm mais peso)

print("Comparando vetorizações:\n")

# CountVectorizer
vectorizer_count = CountVectorizer()
X_train_count = vectorizer_count.fit_transform(X_train)
X_test_count = vectorizer_count.transform(X_test)

nb_count = MultinomialNB()
nb_count.fit(X_train_count, y_train)
acc_count = nb_count.score(X_test_count, y_test)

# TF-IDF Vectorizer
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
acc_tfidf = nb_tfidf.score(X_test_tfidf, y_test)

print(f"CountVectorizer: {acc_count*100:.2f}% - Simples e eficaz")
print(f"TF-IDF Vectorizer: {acc_tfidf*100:.2f}% - Pondera importância das palavras")
print(f"\nNota: CountVectorizer geralmente funciona melhor com Naive Bayes")

<a id='vantagens-desvantagens'></a>
## **Vantagens e Desvantagens do Naive Bayes**

Agora que você conhece o Naive Bayes profundamente, vamos resumir seus pontos fortes e fracos:

### **Vantagens**

1. **Muito rápido:** Extremamente eficiente tanto no treinamento quanto na predição
2. **Simples de implementar:** Fácil de entender e interpretar
3. **Poucos hiperparâmetros:** Praticamente não requer tuning (apenas alpha em alguns casos)
4. **Funciona bem com poucos dados:** Eficaz mesmo com conjuntos de treinamento pequenos
5. **Naturalmente multiclasse:** Sem necessidade de adaptações para múltiplas classes
6. **Robusto a features irrelevantes:** Lida bem com muitas features

### **Desvantagens**

1. **Suposição ingênua de independência:** Raramente verdadeira na prática (ex: "São" e "Paulo" não são independentes)
2. **Não funciona bem com features correlacionadas:** Performance cai quando as features são dependentes
3. **Problema de frequência zero:** Features não vistas no treinamento podem causar problemas (mitigado com smoothing)
4. **Sensível à distribuição:** Gaussian NB assume distribuição normal dos dados
5. **Estimativas de probabilidade não calibradas:** As probabilidades podem não refletir a confiança real
6. **Desempenho limitado em problemas complexos:** Outros algoritmos podem superar em datasets grandes e complexos

### **Quando usar Naive Bayes?**

**Bom para:**
- Classificação de texto (spam, sentimento, categorização)
- Problemas com muitas features
- Baseline rápido para comparação
- Datasets pequenos a médios
- Quando interpretabilidade é importante

**Evitar quando:**
- Features altamente correlacionadas
- Quando probabilidades calibradas são necessárias
- Problemas com relações complexas entre features
- Quando há tempo para treinar modelos mais sofisticados

## **Considerações Importantes**

### **1. Suposição de Independência Condicional**

Embora raramente seja verdadeira em cenários reais, o Naive Bayes frequentemente apresenta boa performance. Isso acontece porque a classificação não exige estimativa de probabilidade perfeitamente calibrada, mas apenas que a classe correta tenha a maior pontuação a posteriori.

### **2. Zero-Frequency Problem**

Se uma feature de uma nova amostra não foi vista durante o treinamento para uma determinada classe, sua probabilidade $P(x_j|c_k)$ seria 0. Devido ao produtório, isso anularia a posteriori para aquela classe. A **suavização de Laplace** (Laplace smoothing) é a técnica usada para mitigar esse problema, adicionando uma contagem mínima a todas as features.

### **3. Estabilidade Numérica (Log-Probabilities)**

A multiplicação de muitas probabilidades pode resultar em valores extremamente pequenos, levando a problemas de underflow (arredondar para 0). Por isso, trabalhamos com a soma dos logaritmos das probabilidades, transformando o produtório em um somatório:

$$\hat{y} = \arg \max_{c_k \in C} \left(\log(P(c_k)) + \sum^d_{j=1}\log(P(x_j|c_k))\right)$$

No código, implementamos isso com `np.log()` para evitar underflow numérico.

### **4. Aplicações do Naive Bayes**

- **Filtragem de spam:** Classificação de emails e mensagens
- **Análise de sentimento:** Determinar se um texto é positivo, negativo ou neutro
- **Categorização de documentos:** Organizar documentos por tópico
- **Diagnóstico médico:** Probabilidade de doenças baseado em sintomas
- **Pontuação de crédito:** Avaliação de credibilidade para empréstimos
- **Previsão do tempo:** Classificação de condições meteorológicas

---

<-- [**Anterior: KNN**](01_knn.ipynb) | [**Próximo: Regressão Logística**](03_regressao_logistica.ipynb) -->